<a href="https://colab.research.google.com/github/Matancoo/IML.HUJI/blob/main/challenge/DATA_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [210]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import pickle
import re

# from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping


import torch
import torchvision
import torchvision.transforms as transforms
import random 
from torchvision import transforms, datasets
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [237]:



def load_data(filename: str):
    """
    Load Agoda booking cancellation dataset
    Parameters
    ----------
    filename: str
        Path to house prices dataset
    Returns
    -------
    Design matrix and response vector in either of the following formats:
    1) Single dataframe with last column representing the response
    2) Tuple of pandas.DataFrame and Series
    3) Tuple of ndarray of shape (n_samples, n_features) and ndarray of shape (n_samples,)
    """
    # Data Preprocessing
    data = pd.read_csv(filename)#.drop_duplicates()

    # reformat the feature names:   #TODO: doesnt work
    # def camel_to_snake(name):
    #     name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    #     return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()
    # # Reformating the feature names: 
    # data.columns = data.columns.map(lambda x : x.strip())


   
    #feature engineering before datetime conversion
    data['is_cancelled'] = (~pd.isna(data.cancellation_datetime)).astype(int) #new response
    


    # creating one hot representation
    # data = pd.get_dummies(data, columns=['accommadation_type_name','charge_option','customer_nationalit'])
#is this right
    # Changing dates to datetime type: 
    data['cancellation_datetime'] = pd.to_datetime(data['cancellation_datetime'])
    data['booking_datetime'] = pd.to_datetime(data['booking_datetime'])
    data['checkout_date'] = pd.to_datetime(data['checkout_date'])
    data['hotel_live_date'] = pd.to_datetime(data['hotel_live_date'])
    data['checkin_date'] = pd.to_datetime(data['checkin_date'])

    #feature engineering after datetime conversion
    data['time_interval'] = abs(data.booking_datetime - data.checkin_date)

    # sample dropping
    data.dropna(subset = ['origin_country_code','origin_country_code'], inplace = True)
    
    # check that check-in is before the end_date
    end_date = pd.to_datetime('2018-12-13')
    data = data[data.checkin_date <= end_date]

    #feature dropping:
    data.drop(columns=['hotel_country_code','hotel_id','h_booking_id'])


    #replaceing nan with 0.0

    data['request_nonesmoke'].replace(to_replace=np.nan, value = 0.0, inplace= True)
    data['request_latecheckin'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_highfloor'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_largebed'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_twinbeds'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_airport'].replace(to_replace=np.nan, value=0.0, inplace = True)
    data['request_earlycheckin'].replace(to_replace=np.nan, value=0.0, inplace = True)

    # data.cancellation_datetime = data['cancellation_datetime'].apply(lambda x: x.date())
    # start_date = pd.to_datetime('2018-12-07')
    # end_date = pd.to_datetime('2018-12-13')
    # df['cancellation_datetime'] = (df.cancellation_datetime <= end_date) &  (df.cancellation_datetime > start_date)






    # Dropping samples that have a checkin-date after 2018-12-13
    #Turning objects to one-hot vectors:
    # labels = data.pop("cancellation_datetime")
    # features = data

    return data





def evaluate_and_export(estimator, X: np.ndarray, filename: str):
    """
    Export to specified file the prediction results of given estimator on given testset.
    File saved is in csv format with a single column named 'predicted_values' and n_samples rows containing
    predicted values.
    Parameters
    ----------
    estimator: BaseEstimator or any object implementing predict() method as in BaseEstimator (for example sklearn)
        Fitted estimator to use for prediction
    X: ndarray of shape (n_samples, n_features)
        Test design matrix to predict its responses
    filename:
        path to store file at
    """
    pd.DataFrame(estimator.predict(X), columns=["predicted_values"]).to_csv(filename, index=False)




Data Preprocessing

In [234]:
from pandas.io.parsers.readers import read_csv
#loading data:
data  = load_data('https://raw.githubusercontent.com/Matancoo/IML.HUJI/main/datasets/agoda_cancellation_train.csv')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


In [235]:
data.columns

Index(['h_booking_id', 'booking_datetime', 'checkin_date', 'checkout_date',
       'hotel_id', 'hotel_country_code', 'hotel_live_date',
       'hotel_star_rating', 'accommadation_type_name', 'charge_option',
       'h_customer_id', 'customer_nationality', 'guest_is_not_the_customer',
       'guest_nationality_country_name', 'no_of_adults', 'no_of_children',
       'no_of_extra_bed', 'no_of_room', 'origin_country_code', 'language',
       'original_selling_amount', 'original_payment_method',
       'original_payment_type', 'original_payment_currency',
       'is_user_logged_in', 'cancellation_policy_code', 'is_first_booking',
       'request_nonesmoke', 'request_latecheckin', 'request_highfloor',
       'request_largebed', 'request_twinbeds', 'request_airport',
       'request_earlycheckin', 'cancellation_datetime', 'hotel_area_code',
       'hotel_brand_code', 'hotel_chain_code', 'hotel_city_code',
       'is_cancelled', 'time_interval'],
      dtype='object')

In [236]:
data

,h_booking_id,booking_datetime,checkin_date,checkout_date,hotel_id,hotel_country_code,hotel_live_date,hotel_star_rating,accommadation_type_name,charge_option,...,request_twinbeds,request_airport,request_earlycheckin,cancellation_datetime,hotel_area_code,hotel_brand_code,hotel_chain_code,hotel_city_code,is_cancelled,time_interval
0,-9223194055642672935,2018-06-28 21:15:00,2018-07-09,2018-07-13,6452,HK,2009-06-28 02:02:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,NaT,1192,NaN,NaN,142,0,873900000000000
1,-9222713784330706132,2018-08-10 22:31:00,2018-08-16,2018-08-17,47729,CN,2011-06-07 11:52:00,4.0,Hotel,Pay Later,...,1.0,0.0,0.0,NaT,5617,902.0,600.0,1273,0,437340000000000
2,-9222411208325704942,2018-09-14 07:55:00,2018-09-14,2018-09-15,780431,KR,2014-11-20 15:43:00,4.0,Hotel,Pay Now,...,0.0,0.0,0.0,NaT,3819,NaN,NaN,2799,0,28500000000000
3,-9222220845872895471,2018-06-25 07:33:00,2018-07-02,2018-07-03,291365,JP,2011-11-21 12:27:00,3.0,Hotel,Pay Later,...,0.0,0.0,0.0,NaT,2379,466.0,236.0,1448,0,577620000000000
4,-9221127186162682116,2018-07-23 10:06:00,2018-08-09,2018-08-10,479046,TH,2013-06-06 10:04:00,3.0,Resort,Pay Later,...,0.0,0.0,0.0,2018-07-23,1506,NaN,NaN,616,1,1432440000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58654,9221958224902563533,2018-07-18 22:07:00,2018-08-04,2018-08-05,2975682,ID,2017-10-12 11:01:00,2.0,Guest House / Bed & Breakfast,Pay Now,...,0.0,0.0,0.0,2018-07-22,995,NaN,NaN,2066,1,1389180000000000
58655,9222015611933681753,2018-09-05 12:07:00,2018-09-06,2018-09-08,3579174,PH,2017-11-30 00:42:00,4.0,Apartment,Pay Now,...,0.0,0.0,0.0,NaT,674,NaN,NaN,2310,0,42780000000000
58656,9222651806661094957,2018-06-08 23:23:00,2018-08-09,2018-08-11,407156,TH,2015-03-14 05:18:00,5.0,Hotel,Pay Now,...,0.0,0.0,0.0,NaT,3823,754.0,55.0,2477,0,5272620000000000
58657,9223221736086584899,2018-09-02 08:18:00,2018-09-10,2018-09-12,303186,DE,2012-04-13 17:48:00,0.0,Hostel,Pay Now,...,1.0,0.0,0.0,NaT,4663,NaN,NaN,1935,0,661320000000000


In [230]:

# # returns a correlation matrix of each feature with all others.
# data = df.select_dtypes(include=np.number)  # select only numerical features
corr_matrix = data.corr(method='pearson').loc[:, ['is_cancelled']].sort_values('is_cancelled')
corr_target = abs(corr_matrix)
corr_target



,is_cancelled
request_largebed,0.084075
request_nonesmoke,0.060085
guest_is_not_the_customer,0.046707
hotel_star_rating,0.033480
hotel_brand_code,0.020096
request_latecheckin,0.019170
is_first_booking,0.014472
hotel_chain_code,0.010988
request_twinbeds,0.010740
request_earlycheckin,0.009130


In [113]:
# Visualizing the percentage of bookings canceled prior to arrival: 
plt.figure(figsize=(8,5)) 
plt.title("Percentage of Canceled vs 'No-Show' Bookings", fontsize = 20, pad = 10)
px.barplot(x=data[data['is_cancelled']==1]['reservation_status'].unique(), y=df[df['is_canceled']==1]['reservation_status'].value_counts(normalize=True), palette=['lightblue', 'lightslategrey'])
plt.xlabel("Booking Status", fontsize = 12, labelpad = 5)
plt.ylabel("Percentage of Bookings", fontsize = 12, labelpad = 5);


23962   2018-06-07
16951   2018-06-15
31122   2018-06-16
18350   2018-06-17
38215   2018-06-21
48650   2018-06-21
37615   2018-06-21
34807   2018-06-21
47352   2018-06-21
2497    2018-06-22
6505    2018-06-22
55629   2018-06-22
51175   2018-06-22
12480   2018-06-23
58459   2018-06-23
38119   2018-06-23
33131   2018-06-23
55683   2018-06-23
15091   2018-06-23
46389   2018-06-23
Name: checkin_date, dtype: datetime64[ns]

In [ ]:
# if __name__ == '__main__':
#     np.random.seed(0)

#     # Load data
#     df, cancellation_labels = load_data("../datasets/agoda_cancellation_train.csv")
#     train_X, train_y, test_X, test_y = split_train_test(df, cancellation_labels)

#     # Fit model over data
#     estimator = AgodaCancellationEstimator().fit(train_X, train_y)

#     # Store model predictions over test set
#     evaluate_and_export(estimator, test_X, "id1_id2_id3.csv")